In [1]:
import numpy as np, pandas as pd
import os,glob, re
%matplotlib inline

dfs = {re.search('([a-zA-Z_]*)\.csv', fn).group(1):pd.read_csv(fn) for fn in glob.glob(os.getcwd()+'//Raw//*.csv')}
print('data frames read:{}'.format(list(dfs.keys())))

print('local variables with the same names are created.')
for k, v in dfs.items(): locals()[k] = v

data frames read:['air_reserve', 'air_store_info', 'air_visit_data', 'date_info', 'hpg_reserve', 'hpg_store_info', 'sample_submission', 'store_id_relation']
local variables with the same names are created.


In [2]:
print("Raw shape of each dataset")
for k, v in dfs.items(): print("%s : "%k,v.shape)

Raw shape of each dataset
air_reserve :  (92378, 4)
air_store_info :  (829, 5)
air_visit_data :  (252108, 3)
date_info :  (517, 3)
hpg_reserve :  (2000320, 4)
hpg_store_info :  (4690, 5)
sample_submission :  (32019, 2)
store_id_relation :  (150, 2)


In [3]:
print("Split id column in sample_submission")
sample_submission["air_store_id"],sample_submission["visit_date"] = sample_submission.id.str[:20],sample_submission.id.str[21:]
sample_submission.head()

Split id column in sample_submission


,id,visitors,air_store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27


In [4]:
print("Unique store Ids in each dataset")
for k, v in dfs.items(): 
    try:       
        print(k," - Unqiue air_stores: ",v.air_store_id.nunique())
    except:
        pass
    try:
        print(k," - Unqiue hpg_stores: ",v.hpg_store_id.nunique())
    except:
        pass

Unique store Ids in each dataset
air_reserve  - Unqiue air_stores:  314
air_store_info  - Unqiue air_stores:  829
air_visit_data  - Unqiue air_stores:  829
hpg_reserve  - Unqiue hpg_stores:  13325
hpg_store_info  - Unqiue hpg_stores:  4690
sample_submission  - Unqiue air_stores:  821
store_id_relation  - Unqiue air_stores:  150
store_id_relation  - Unqiue hpg_stores:  150


In [5]:
air_reserve['visit_date'] = pd.to_datetime(air_reserve['visit_datetime']).dt.date.astype(str)

In [6]:
reserve_summary = air_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors'].sum().reset_index()

In [7]:
reserve_summary.head()

,air_store_id,visit_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-05,9
2,air_00a91d42b08b08d9,2016-12-14,18
3,air_00a91d42b08b08d9,2016-12-17,2
4,air_00a91d42b08b08d9,2016-12-20,4


In [8]:
new_train = air_visit_data.merge(reserve_summary, on =['air_store_id','visit_date'],how = 'left').fillna(0)

In [9]:
new_train.head()

,air_store_id,visit_date,visitors,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,0.0
1,air_ba937bf13d40fb24,2016-01-14,32,0.0
2,air_ba937bf13d40fb24,2016-01-15,29,0.0
3,air_ba937bf13d40fb24,2016-01-16,22,0.0
4,air_ba937bf13d40fb24,2016-01-18,6,0.0


In [10]:
new_train['walkins'] = new_train['visitors'] - new_train['reserve_visitors']
new_train.loc[new_train['walkins'] <0,'walkins'] = 0
new_train['noshows'] = new_train['reserve_visitors'] - new_train['visitors']
new_train.loc[new_train['noshows'] <0,'noshows'] = 0
new_train.head()

,air_store_id,visit_date,visitors,reserve_visitors,walkins,noshows
0,air_ba937bf13d40fb24,2016-01-13,25,0.0,25.0,0.0
1,air_ba937bf13d40fb24,2016-01-14,32,0.0,32.0,0.0
2,air_ba937bf13d40fb24,2016-01-15,29,0.0,29.0,0.0
3,air_ba937bf13d40fb24,2016-01-16,22,0.0,22.0,0.0
4,air_ba937bf13d40fb24,2016-01-18,6,0.0,6.0,0.0


In [11]:
weekdayholidays = date_info.apply(lambda x: x.day_of_week in ['Saturday','Sunday'] and x.holiday_flg == 1,axis=1)
date_info.loc[weekdayholidays,'holiday_flg'] = 0
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,0
2,2016-01-03,Sunday,0
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [12]:
date_info['weights'] = ((date_info.index + 1)/ len(date_info))**7
date_info.tail()

,calendar_date,day_of_week,holiday_flg,weights
512,2017-05-27,Saturday,0,0.947082
513,2017-05-28,Sunday,0,0.960081
514,2017-05-29,Monday,0,0.973233
515,2017-05-30,Tuesday,0,0.986539
516,2017-05-31,Wednesday,0,1.000000


In [13]:
new_train = new_train.merge(date_info,left_on = 'visit_date',right_on = 'calendar_date', how ='left').drop('calendar_date',axis = 1)

In [14]:
new_train.head()

,air_store_id,visit_date,visitors,reserve_visitors,walkins,noshows,day_of_week,holiday_flg,weights
0,air_ba937bf13d40fb24,2016-01-13,25,0.0,25.0,0.0,Wednesday,0,6.355792e-12
1,air_ba937bf13d40fb24,2016-01-14,32,0.0,32.0,0.0,Thursday,0,1.067732e-11
2,air_ba937bf13d40fb24,2016-01-15,29,0.0,29.0,0.0,Friday,0,1.730633e-11
3,air_ba937bf13d40fb24,2016-01-16,22,0.0,22.0,0.0,Saturday,0,2.718980e-11
4,air_ba937bf13d40fb24,2016-01-18,6,0.0,6.0,0.0,Monday,0,6.201171e-11


In [15]:
new_train['visitors'] = new_train['visitors'].apply(pd.np.log1p)
new_train['reserve_visitors'] = new_train['reserve_visitors'].apply(pd.np.log1p)
new_train['walkins'] = new_train['walkins'].apply(pd.np.log1p)
new_train['noshows'] = new_train['noshows'].apply(pd.np.log1p)
new_train.tail()

,air_store_id,visit_date,visitors,reserve_visitors,walkins,noshows,day_of_week,holiday_flg,weights
252103,air_24e8414b9b07decb,2017-04-18,1.945910,0.0,1.945910,0.0,Tuesday,0,0.544521
252104,air_24e8414b9b07decb,2017-04-19,1.945910,0.0,1.945910,0.0,Wednesday,0,0.552613
252105,air_24e8414b9b07decb,2017-04-20,2.079442,0.0,2.079442,0.0,Thursday,0,0.560809
252106,air_24e8414b9b07decb,2017-04-21,2.197225,0.0,2.197225,0.0,Friday,0,0.569108
252107,air_24e8414b9b07decb,2017-04-22,1.791759,0.0,1.791759,0.0,Saturday,0,0.577513


In [20]:
weighted_mean_visitors = lambda x : ((x.visitors * x.weights).sum() / (x.weights).sum())
visitors_per_weekday = new_train.groupby(['air_store_id','day_of_week','holiday_flg']).apply(weighted_mean_visitors).reset_index()
visitors_per_weekday.head()

,air_store_id,day_of_week,holiday_flg,0
0,air_00a91d42b08b08d9,Friday,0,3.585544
1,air_00a91d42b08b08d9,Monday,0,3.228023
2,air_00a91d42b08b08d9,Monday,1,3.091042
3,air_00a91d42b08b08d9,Saturday,0,2.499792
4,air_00a91d42b08b08d9,Sunday,0,1.098612


In [21]:
weighted_mean_reservations = lambda x : ((x.reserve_visitors * x.weights).sum() / (x.weights).sum())
reserves_per_weekday = new_train.groupby(['air_store_id','day_of_week','holiday_flg']).apply(weighted_mean_reservations).reset_index()
reserves_per_weekday.head()

,air_store_id,day_of_week,holiday_flg,0
0,air_00a91d42b08b08d9,Friday,0,0.082399
1,air_00a91d42b08b08d9,Monday,0,0.036146
2,air_00a91d42b08b08d9,Monday,1,0.000000
3,air_00a91d42b08b08d9,Saturday,0,0.129386
4,air_00a91d42b08b08d9,Sunday,0,0.000000


In [22]:
weighted_mean_walkins = lambda x : ((x.walkins * x.weights).sum() / (x.weights).sum())
walkin_visitors_per_weekday = new_train.groupby(['air_store_id','day_of_week','holiday_flg']).apply(weighted_mean_walkins).reset_index()
walkin_visitors_per_weekday.head()

,air_store_id,day_of_week,holiday_flg,0
0,air_00a91d42b08b08d9,Friday,0,3.581131
1,air_00a91d42b08b08d9,Monday,0,3.223614
2,air_00a91d42b08b08d9,Monday,1,3.091042
3,air_00a91d42b08b08d9,Saturday,0,2.466632
4,air_00a91d42b08b08d9,Sunday,0,1.098612


In [23]:
weighted_mean_noshows = lambda x : ((x.noshows * x.weights).sum() / (x.weights).sum())
noshows_per_weekday = new_train.groupby(['air_store_id','day_of_week','holiday_flg']).apply(weighted_mean_noshows).reset_index()
noshows_per_weekday.head()

,air_store_id,day_of_week,holiday_flg,0
0,air_00a91d42b08b08d9,Friday,0,0.0
1,air_00a91d42b08b08d9,Monday,0,0.0
2,air_00a91d42b08b08d9,Monday,1,0.0
3,air_00a91d42b08b08d9,Saturday,0,0.0
4,air_00a91d42b08b08d9,Sunday,0,0.0


In [61]:
summarized_train = visitors_per_weekday.merge(
    reserves_per_weekday, on= ['air_store_id','day_of_week','holiday_flg'],how = 'outer')

summarized_train.rename(columns={'0_x':'wt_visitors','0_y':'wt_reserves'},inplace = True)

summarized_train = summarized_train.merge(
    walkin_visitors_per_weekday, on = ['air_store_id','day_of_week','holiday_flg'],how ='outer')

summarized_train = summarized_train.merge(
    noshows_per_weekday, on= ['air_store_id','day_of_week','holiday_flg'],how = 'outer')

summarized_train.rename(columns={'0_x':'walkins','0_y':'noshows'},inplace = True)

summarized_train.head()

,air_store_id,day_of_week,holiday_flg,wt_visitors,wt_reserves,walkins,noshows
0,air_00a91d42b08b08d9,Friday,0,3.585544,0.082399,3.581131,0.0
1,air_00a91d42b08b08d9,Monday,0,3.228023,0.036146,3.223614,0.0
2,air_00a91d42b08b08d9,Monday,1,3.091042,0.000000,3.091042,0.0
3,air_00a91d42b08b08d9,Saturday,0,2.499792,0.129386,2.466632,0.0
4,air_00a91d42b08b08d9,Sunday,0,1.098612,0.000000,1.098612,0.0


In [72]:
test = sample_submission.merge(date_info,left_on='visit_date',right_on='calendar_date',how = 'left').drop(['calendar_date','weights'],axis = 1)
test.head()

,id,visitors,air_store_id,visit_date,day_of_week,holiday_flg
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0


In [108]:
newtest = test.merge(reserve_summary, on=['air_store_id','visit_date'], how='left').fillna(0)
newtest.head()

,id,visitors,air_store_id,visit_date,day_of_week,holiday_flg,reserve_visitors
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0,0.0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0,0.0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,0.0
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,0.0
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0,0.0


In [109]:
newtest = newtest.merge(summarized_train,on = ['air_store_id','day_of_week','holiday_flg'], how = 'left')
newtest.head()

,id,visitors,air_store_id,visit_date,day_of_week,holiday_flg,reserve_visitors,wt_visitors,wt_reserves,walkins,noshows
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0,0.0,1.098612,0.000000,1.098612,0.0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0,0.0,3.228023,0.036146,3.223614,0.0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,0.0,3.370217,0.341207,3.339184,0.0
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,0.0,3.361901,0.112714,3.340719,0.0
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0,0.0,3.495896,0.121620,3.479477,0.0


In [110]:
newtest.loc[newtest.wt_visitors.isnull()].shape

(668, 11)

In [111]:
temp = newtest[newtest.wt_visitors.isnull()].merge(summarized_train[summarized_train.holiday_flg == 0]
                                            ,on = ['air_store_id','day_of_week'], how = 'left')


newtest.loc[newtest.wt_visitors.isnull(),'wt_visitors'] = temp['wt_visitors_y'].values
newtest.loc[newtest.wt_reserves.isnull(),'wt_reserves'] = temp['wt_reserves_y'].values
newtest.loc[newtest.walkins.isnull(),'walkins'] = temp['walkins_y'].values
newtest.loc[newtest.noshows.isnull(),'noshows'] = temp['noshows_y'].values
newtest.shape

(32019, 11)

In [112]:
newtest.loc[newtest.wt_visitors.isnull()].shape

(448, 11)

In [113]:
temp2 = newtest[newtest.wt_visitors.isnull()].merge(summarized_train[[
    'air_store_id','wt_visitors','wt_reserves','walkins','noshows']].groupby('air_store_id').mean().reset_index(),
                                                    on = 'air_store_id',how = "left")

newtest.loc[newtest.wt_visitors.isnull(),'wt_visitors'] = temp2['wt_visitors_y'].values
newtest.loc[newtest.wt_reserves.isnull(),'wt_reserves'] = temp2['wt_reserves_y'].values
newtest.loc[newtest.walkins.isnull(),'walkins'] = temp2['walkins_y'].values
newtest.loc[newtest.noshows.isnull(),'noshows'] = temp2['noshows_y'].values
newtest.shape

(32019, 11)

In [114]:
newtest.loc[newtest.wt_visitors.isnull()].shape

(0, 11)

In [115]:
max_visitors = air_visit_data.groupby('air_store_id')['visitors'].max().reset_index()
max_visitors.rename(columns = {'visitors' : 'max_cap'},inplace = True)
max_visitors.head()

,air_store_id,max_cap
0,air_00a91d42b08b08d9,99
1,air_0164b9927d20bcc3,27
2,air_0241aa3964b7f861,48
3,air_0328696196e46f18,41
4,air_034a3d5b40d5b1b1,116


In [116]:
newtest = newtest.merge(max_visitors,on= 'air_store_id', how = 'left')
newtest.head()

,id,visitors,air_store_id,visit_date,day_of_week,holiday_flg,reserve_visitors,wt_visitors,wt_reserves,walkins,noshows,max_cap
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0,0.0,1.098612,0.000000,1.098612,0.0,99
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0,0.0,3.228023,0.036146,3.223614,0.0,99
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,0.0,3.370217,0.341207,3.339184,0.0,99
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,0.0,3.361901,0.112714,3.340719,0.0,99
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0,0.0,3.495896,0.121620,3.479477,0.0,99


In [117]:
newtest.drop(['visitors','air_store_id','visit_date','day_of_week','holiday_flg'],inplace = True,axis = 1)
newtest.head()

,id,reserve_visitors,wt_visitors,wt_reserves,walkins,noshows,max_cap
0,air_00a91d42b08b08d9_2017-04-23,0.0,1.098612,0.000000,1.098612,0.0,99
1,air_00a91d42b08b08d9_2017-04-24,0.0,3.228023,0.036146,3.223614,0.0,99
2,air_00a91d42b08b08d9_2017-04-25,0.0,3.370217,0.341207,3.339184,0.0,99
3,air_00a91d42b08b08d9_2017-04-26,0.0,3.361901,0.112714,3.340719,0.0,99
4,air_00a91d42b08b08d9_2017-04-27,0.0,3.495896,0.121620,3.479477,0.0,99


In [118]:
newtest['wt_visitors'] = newtest['wt_visitors'].apply(pd.np.expm1)
newtest['wt_reserves'] = newtest['wt_reserves'].apply(pd.np.expm1)
newtest['walkins'] = newtest['walkins'].apply(pd.np.expm1)
newtest['noshows'] = newtest['noshows'].apply(pd.np.expm1)
newtest.head()

,id,reserve_visitors,wt_visitors,wt_reserves,walkins,noshows,max_cap
0,air_00a91d42b08b08d9_2017-04-23,0.0,2.000000,0.000000,2.000000,0.0,99
1,air_00a91d42b08b08d9_2017-04-24,0.0,24.229717,0.036808,24.118737,0.0,99
2,air_00a91d42b08b08d9_2017-04-25,0.0,28.084839,0.406644,27.196102,0.0,99
3,air_00a91d42b08b08d9_2017-04-26,0.0,27.843975,0.119312,27.239433,0.0,99
4,air_00a91d42b08b08d9_2017-04-27,0.0,31.979835,0.129325,31.442758,0.0,99


In [119]:
newtest['calculated_visits'] = ((newtest['reserve_visitors']+newtest['wt_reserves'])/2) +newtest['walkins'] - newtest['noshows']

In [145]:
k = .6

newtest['visitors'] = ((newtest['wt_visitors'] * k) + ((1-k)*newtest['calculated_visits']))
newtest.head()

,id,reserve_visitors,wt_visitors,wt_reserves,walkins,noshows,max_cap,calculated_visits,visitors
0,air_00a91d42b08b08d9_2017-04-23,0.0,2.000000,0.000000,2.000000,0.0,99,2.000000,2.000000
1,air_00a91d42b08b08d9_2017-04-24,0.0,24.229717,0.036808,24.118737,0.0,99,24.137141,24.192686
2,air_00a91d42b08b08d9_2017-04-25,0.0,28.084839,0.406644,27.196102,0.0,99,27.399424,27.810673
3,air_00a91d42b08b08d9_2017-04-26,0.0,27.843975,0.119312,27.239433,0.0,99,27.299089,27.626021
4,air_00a91d42b08b08d9_2017-04-27,0.0,31.979835,0.129325,31.442758,0.0,99,31.507420,31.790869


In [146]:
newtest.loc[newtest['visitors'] > newtest['max_cap'],'visitors'] = newtest['max_cap']

In [147]:
newtest.loc[newtest['visitors'] < 0,'visitors'] = newtest['wt_reserves']

In [148]:
result = newtest[['id','visitors']]

In [149]:
result.shape

(32019, 2)

In [150]:
result.to_csv('result_dump4.csv', float_format='%.4f', index=None)

In [135]:
newtest.describe()

,reserve_visitors,wt_visitors,wt_reserves,walkins,noshows,max_cap,calculated_visits,visitors
count,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000
mean,0.366813,18.784779,1.632487,16.489033,0.021708,76.069428,17.466975,9.261401
std,3.891000,14.273438,4.718326,13.517187,0.258801,69.372377,13.695208,7.040467
min,0.000000,1.000000,0.000000,0.000000,0.000000,10.000000,-7.500000,0.500000
25%,0.000000,8.146902,0.000000,6.594211,0.000000,40.000000,7.416091,4.001858
50%,0.000000,15.185882,0.000000,12.663357,0.000000,61.000000,13.781842,7.478057
75%,0.000000,25.594665,0.238433,22.876529,0.000000,90.000000,23.869239,12.566069
max,511.000000,182.000000,90.000000,141.370436,24.000000,877.000000,280.390355,86.500000


In [136]:
result.describe()

,visitors
count,32019.000000
mean,9.261401
std,7.040467
min,0.500000
25%,4.001858
50%,7.478057
75%,12.566069
max,86.500000
